In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dataloaders
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
train_dataset_path = "train_data"
test_dataset_path = "test_data"
image_size = (64,64)

In [34]:
# Setting device
def set_device():
  if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda")
  else:
    device = torch.device("cpu")
    print("cpu")
  return device

In [35]:
transform_dataset = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(train_dataset_path,transform_dataset)
train_loader = dataloaders.DataLoader(train_dataset,32,False)
mean = 0.
std = 0.
total_image_count = 0
for image, _ in train_loader:
    image = image.view(image.size(0), image.size(1), -1)
    mean += image.mean(2).sum(0)
    std += image.std(2).sum(0)
    total_image_count += image.size(0)

mean /= total_image_count
std /= total_image_count

In [36]:
# train data transformation
train_transform_dataset = transforms.Compose([transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
train_dataset = datasets.ImageFolder(train_dataset_path,train_transform_dataset)
train_loader = dataloaders.DataLoader(train_dataset,32,True)

# test data transformation
test_transform_dataset = transforms.Compose([transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
test_dataset = datasets.ImageFolder(test_dataset_path,test_transform_dataset)
test_loader = dataloaders.DataLoader(test_dataset,32,False)

In [ ]:
trained_model = models.resnet50(pretrained=False)
loss_fn = nn.CrossEntropyLoss()
num_features = trained_model.fc.in_features
number_of_classes = len(train_dataset.classes)
trained_model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Linear(512, number_of_classes)
)
device = set_device()
trained_model = trained_model.to(device)
optimizer = optim.AdamW(trained_model.parameters(),lr=1e-4, weight_decay=1e-4)

cpu


In [ ]:
num_epochs = 25
device = set_device()

for epoch in range(num_epochs):
    trained_model.train()
    running_loss = 0.0
    running_correct = 0.0
    total = 0
    for data in train_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        total += labels.size(0)
        optimizer.zero_grad()
        outputs = trained_model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_correct += (predicted == labels).sum().item()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100.0 * (running_correct / total)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

In [ ]:
torch.save(trained_model.state_dict(), 'model_complete.pth')